# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298199221386                   -0.85    5.1         
  2   -8.300165577685       -2.71       -1.26    1.0   83.1ms
  3   -8.300439839864       -3.56       -1.89    3.1    112ms
  4   -8.300461467676       -4.66       -2.76    1.6   95.0ms
  5   -8.300463846573       -5.62       -3.04    2.8    218ms
  6   -8.300464244393       -6.40       -3.23    1.6   80.7ms
  7   -8.300464442004       -6.70       -3.37    2.2   94.7ms
  8   -8.300464546743       -6.98       -3.51    4.6    113ms
  9   -8.300464624473       -7.11       -3.73    6.8    175ms
 10   -8.300464627581       -8.51       -3.79    1.1    101ms
 11   -8.300464641888       -7.84       -4.07    3.1    155ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67459867872                   -0.70    7.2         
  2   -16.67863356533       -2.39       -1.14    1.0    185ms
  3   -16.67921697832       -3.23       -1.87    2.5    250ms
  4   -16.67927458771       -4.24       -2.77    3.1    223ms
  5   -16.67928592293       -4.95       -3.19    5.4    322ms
  6   -16.67928619891       -6.56       -3.48    2.2    237ms
  7   -16.67928621647       -7.76       -3.97    1.2    176ms
  8   -16.67928622141       -8.31       -4.56    2.9    235ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31793470467                   -0.56    6.9         
  2   -33.32581739915       -2.10       -1.00    1.4    697ms
  3   -33.33595716585       -1.99       -1.68    5.6    937ms
  4   -33.33616208975       -3.69       -2.53    3.2    785ms
  5   -33.33694018742       -3.11       -2.90    8.5    1.19s
  6   -33.33694308147       -5.54       -3.27    4.2    859ms
  7   -33.33694376183       -6.17       -4.03    2.6    788ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298433715927                   -0.85    4.9         
  2   -8.300271153254       -2.74       -1.59    1.8   66.9ms
  3   -8.300435243751       -3.78       -2.65    1.4   63.4ms
  4   -8.300423160647   +   -4.92       -2.44    5.2    142ms
  5   -8.300464403825       -4.38       -3.38    1.0   64.5ms
  6   -8.300464589863       -6.73       -3.87    2.6   91.4ms
  7   -8.300464639734       -7.30       -4.26    1.5   79.8ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32768879344                   -0.56    7.2         
  2   -33.32746690656   +   -3.65       -1.27    1.5    600ms
  3   -22.91385032749   +    1.02       -0.56    6.2    1.16s
  4   -33.28086168865        1.02       -1.60    5.4    1.10s
  5   -33.14736371947   +   -0.87       -1.27    3.5    880ms
  6   -30.90034286450   +    0.35       -0.87    4.9    1.01s
  7   -33.33413324909        0.39       -2.14    4.5    928ms
  8   -33.33279314643   +   -2.87       -2.12    2.5    825ms
  9   -33.33613981562       -2.48       -2.49    1.9    674ms
 10   -33.33584231114   +   -3.53       -2.51    1.4    587ms
 11   -33.33658888568       -3.13       -2.73    1.8    640ms
 12   -33.33691662057       -3.48      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.